In [6]:
import nltk
from nltk.text import Text
from bs4 import BeautifulSoup
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from prettytable import PrettyTable
import re
import pickle
import math
import numpy as np
import os

In [2]:
#Convertimos en un Pk los objetos
def serializeObject(obj, fname="lemmas2.pkl"):
    with open(fname, "wb") as f:
        pickle.dump(obj, f, -1)
#desconvertimos los pkl
def deserializeObject(fname="lemmas2.pkl"):
    with open(fname, "rb") as f:
        return pickle.load(f)

In [3]:
def get_articles(fname):
        
    f=open(fname, encoding='utf-8')
    text_string=f.read()
    f.close()
    list_articles = text_string.split('<h3>')
    return list_articles
    

In [4]:
def get_text_string(articles):
    '''Receives an html file with a Spanish text, deletes html tags.
    Returns text as a string.'''
    list_articles = []
    for text_string in articles:
        soup = BeautifulSoup(text_string, 'lxml')
        text_string = soup.get_text()
        list_articles.append(text_string)
    return list_articles

In [5]:
def tagSentencesSpanishTagger(sentences):
    sentences_tagged = []
    spanish_tagger = deserializeObject("tagger.pkl")#agarramos nuestro tagger entrenado
    for s in sentences:
        tokens = nltk.word_tokenize(s)#Tokenizamos los articulos
        s_tagged = spanish_tagger.tag(tokens)#Taggeamos los tokens
        s_tagged = [(it[0].lower(), it[1][0].lower()) for it in s_tagged]#Pasamos a minusculas
        sentences_tagged.append(lemmatize(removeStopWords(clearTokens(s_tagged))))#Agregamos las oraciones taggeadas
    return sentences_tagged
#Sacamos las palabras de las oraciones de todas las sentencias
def getWordsFromSentences(sentences):
    words = []
    for sentence in sentences:
        for word in sentence:
            words.append(word)
    return words
#Limpiamos los tokens

def removeStopWords(wordsWithTag, language='spanish'):
    '''Receives a list of words and returns another list without stop words'''
    return [ word for word in wordsWithTag if word[0] not in stopwords.words(language) ]


def clearTokens(tokensWithTag):
    '''Receives a list of  with tag and returns another list with the same tokens but only with letters'''
    result = []
    for token in tokensWithTag:
        clearToken = ""
        for c in token[0]:
            if re.match(r'[a-záéíóúñüA-ZÁÉÍÓÚÑÜ]', c):
                clearToken += c
        if len(clearToken) > 0:
            result.append((clearToken, token[1]))
    return result

def lemmatize(wordsTagged, fname="lemmas2.pkl"):
    lemmas = deserializeObject(fname)
    wordsLemmatized = []
    for word in wordsTagged:
        if word in lemmas.keys():#Si la palabra la encuentra en el dic
            wordsLemmatized.append((lemmas[word], word[1]))#Guardamos el lemma con su tag y la agregamos a las palabras lematizadas
        else:
            wordsLemmatized.append(word)#Si no la encuentra guarda la palabra en el vocabulario en el diccionario
    return wordsLemmatized


In [13]:
if __name__=='__main__':
    #Cadena de texto
    #---------------PRIMERA PARTE--------------#
    fname='text.htm'#Cambiamos por txt despues de procesar htm------2
    articles = get_articles(fname)
    #articles = get_text_string(articles)
    #articlesWithTags = tagSentencesSpanishTagger(articles)
    #serializeObject(articlesWithTags,'articlesTagged.pkl')
    topicos = (['gobierno','n'],['política','n'],['internet','n'],['méxico','n'],['mercado','n'])
    articlesWithTags = deserializeObject('articlesTagged.pkl')
    list_type_articles = []
    
    for articulo in articlesWithTags:
        topics =[]
        aux = 0
        for topico in topicos:
            line = []
            count = 0
            for word in articulo:
                if word == tuple(topico):
                    count +=1
            aux = aux+count
            line.append(topico)
            line.append(count)
            topics.append(line)
        topics2 = []
        if aux != 0 :
            numbers = [x[1] / aux for x in topics]
            for i,topico in enumerate(topicos):
                list_a = []
                list_a.append(topico)
                list_a.append(numbers[i])
                topics2.append(list_a)
        else:
            topics2 = topics
        #topics2.sort(key = lambda x: x[1], reverse=True)
        list_type_articles.append(topics2)
    print(len(list_type_articles))
    
    x = PrettyTable()
    x.field_names = ['N°',topicos[0],topicos[1], topicos[2], topicos[3], topicos[4]]
    for i,par in enumerate(list_type_articles):
        x.add_row([i,par[0][1],par[1][1],par[2][1],par[3][1],par[4][1]])
    print(x)

63
+----+----------------------+---------------------+--------------------+----------------------+---------------------+
| N° |  ['gobierno', 'n']   |  ['política', 'n']  | ['internet', 'n']  |   ['méxico', 'n']    |   ['mercado', 'n']  |
+----+----------------------+---------------------+--------------------+----------------------+---------------------+
| 0  |          0           |          0          |         0          |          0           |          0          |
| 1  |          0           |          0          |         0          |          0           |          0          |
| 2  |         1.0          |         0.0         |        0.0         |         0.0          |         0.0         |
| 3  |         0.0          |         0.5         |        0.0         |         0.0          |         0.5         |
| 4  |         0.0          |         0.0         |        0.0         |         1.0          |         0.0         |
| 5  |         0.0          |         0.0         |  